<a href="https://colab.research.google.com/github/LIZZETHGOMEZ/BEDU-Santander-2021/blob/main/API's_y_concatenacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sesión 6: **Manejo de API'S**

### Notas sobre solicitud de información de HTTP
- GET: Lo usamos cuando queremos pedir información
- POST: Lo usamos cuando queremos enviar información para crear algo (una cuenta de usuario, por ejemplo)
- PUT: Lo usamos cuando queremos sustituir algún dato por otro
- PATCH: Lo usamos cuando queremos modificar algún dato
- DELETE: Lo usamos cuando queremos eliminar algún dato

In [1]:
#Instalamos biblioteca para peticiones información en HTTP
!pip install requests

In [26]:
import requests
import pandas as pd

In [7]:
#Por convención les llamamos endpoint a los accesos
endpoint = " https://api.nasa.gov/neo/rest/v1/neo/browse/"

payload = {"api_key": "NGkLULRS1fZXFUD26ZKmffmwkLhhWhfFW43QlLip"}

In [22]:
# Llamamos al archivo con los parametros de el acceso y la llave de acceso
r = requests.get(endpoint, payload)

# Veamos el estatus, con 200 sabemos que el request fue exitoso.
r.status_code

200

In [ ]:
#Observemos los datos que estan en formato json
r.json()

In [32]:
json = r.json()
json.keys()

dict_keys(['links', 'page', 'near_earth_objects'])

In [33]:
json["links"]

{'next': 'http://www.neowsapp.com/rest/v1/neo/browse?page=1&size=20&api_key=NGkLULRS1fZXFUD26ZKmffmwkLhhWhfFW43QlLip',
 'self': 'http://www.neowsapp.com/rest/v1/neo/browse?page=0&size=20&api_key=NGkLULRS1fZXFUD26ZKmffmwkLhhWhfFW43QlLip'}

In [34]:
json["page"]

{'number': 0, 'size': 20, 'total_elements': 26326, 'total_pages': 1317}

In [ ]:
json['near_earth_objects'] # Observemos que en realidad es una lista

In [ ]:
# Con pprint obtenemos una visualizacion mas organizada de los datos
from pprint import pprint
pprint(json['near_earth_objects'][0])

In [43]:
# Guardamos y transformamos a df
data = json["near_earth_objects"]
df = pd.DataFrame(data)
df.head() #Notemos que esta feo seguimos teniendo un estilo json y lo corregimos normalizando

,links,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,estimated_diameter,is_potentially_hazardous_asteroid,close_approach_data,orbital_data,is_sentry_object
0,{'self': 'http://www.neowsapp.com/rest/v1/neo/...,2000433,2000433,433 Eros (A898 PA),Eros,433,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000433,10.42,{'kilometers': {'estimated_diameter_min': 21.9...,False,"[{'close_approach_date': '1900-12-27', 'close_...","{'orbit_id': '659', 'orbit_determination_date'...",False
1,{'self': 'http://www.neowsapp.com/rest/v1/neo/...,2000719,2000719,719 Albert (A911 TB),Albert,719,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000719,15.51,{'kilometers': {'estimated_diameter_min': 2.10...,False,"[{'close_approach_date': '1909-08-21', 'close_...","{'orbit_id': '220', 'orbit_determination_date'...",False
2,{'self': 'http://www.neowsapp.com/rest/v1/neo/...,2000887,2000887,887 Alinda (A918 AA),Alinda,887,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000887,13.87,{'kilometers': {'estimated_diameter_min': 4.47...,False,"[{'close_approach_date': '1910-01-04', 'close_...","{'orbit_id': '427', 'orbit_determination_date'...",False
3,{'self': 'http://www.neowsapp.com/rest/v1/neo/...,2001036,2001036,1036 Ganymed (A924 UB),Ganymed,1036,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001036,9.25,{'kilometers': {'estimated_diameter_min': 37.5...,False,"[{'close_approach_date': '1910-02-25', 'close_...","{'orbit_id': '925', 'orbit_determination_date'...",False
4,{'self': 'http://www.neowsapp.com/rest/v1/neo/...,2001221,2001221,1221 Amor (1932 EA1),Amor,1221,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001221,17.39,{'kilometers': {'estimated_diameter_min': 0.88...,False,"[{'close_approach_date': '1900-03-08', 'close_...","{'orbit_id': '105', 'orbit_determination_date'...",False


In [44]:
# Normalizamos el df
# Desanidamos el diccionario y colocamos los diccionarios dentro de otros diccionarios como columnas
normalized = pd.json_normalize(data)
df = pd.DataFrame.from_dict(normalized)
df.head()


,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max,orbital_data.orbit_id,orbital_data.orbit_determination_date,orbital_data.first_observation_date,orbital_data.last_observation_date,orbital_data.data_arc_in_days,orbital_data.observations_used,orbital_data.orbit_uncertainty,orbital_data.minimum_orbit_intersection,orbital_data.jupiter_tisserand_invariant,orbital_data.epoch_osculation,orbital_data.eccentricity,orbital_data.semi_major_axis,orbital_data.inclination,orbital_data.ascending_node_longitude,orbital_data.orbital_period,orbital_data.perihelion_distance,orbital_data.perihelion_argument,orbital_data.aphelion_distance,orbital_data.perihelion_time,orbital_data.mean_anomaly,orbital_data.mean_motion,orbital_data.equinox,orbital_data.orbit_class.orbit_class_type,orbital_data.orbit_class.orbit_class_description,orbital_data.orbit_class.orbit_class_range
0,2000433,2000433,433 Eros (A898 PA),Eros,433,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000433,10.42,False,"[{'close_approach_date': '1900-12-27', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/2000433?ap...,21.905591,48.982391,21905.591097,48982.390780,13.611499,30.436237,71868.739495,160703.386968,659,2021-05-24 17:55:05,1893-10-29,2021-05-13,46582,9130,0,.149241,4.582,2459396.5,.2229136604081975,1.458124977590459,10.83050401540394,304.2987825663975,643.1178311831666,1.133089001503149,178.8630846657683,1.783160953677769,2459159.314278333564,132.7701638171466,.5597730035531674,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
1,2000719,2000719,719 Albert (A911 TB),Albert,719,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000719,15.51,False,"[{'close_approach_date': '1909-08-21', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/2000719?ap...,2.101624,4.699374,2101.623793,4699.373665,1.305888,2.920055,6895.091406,15417.893094,220,2021-07-02 05:52:54,1911-10-04,2021-04-17,40008,1892,0,.201652,3.140,2459396.5,.5468676237676214,2.638030362514431,11.57324983811486,183.8547682318951,1565.013584129889,1.195376966739327,156.2074111005377,4.080683758289534,2459956.001930185700,231.2977977256108,.2300299522321089,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
2,2000887,2000887,887 Alinda (A918 AA),Alinda,887,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2000887,13.87,False,"[{'close_approach_date': '1910-01-04', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/2000887?ap...,4.472548,10.000921,4472.547654,10000.920588,2.779111,6.214282,14673.713246,32811.420301,427,2021-08-01 05:51:49,1918-02-03,2021-07-31,37799,2624,0,.0821373,3.221,2459396.5,.5704587918843774,2.473668671636058,9.393753932071414,110.4327677798952,1421.054454062883,1.06254262969232,350.5075830020164,3.884794713579797,2459258.738806992726,34.89945747034967,.2533330084366138,J2000,AMO,Near-Earth asteroid orbits similar to that of ...,1.017 AU < q (perihelion) < 1.3 AU
3,2001036,2001036,1036 Ganymed (A924 UB),Ganymed,1036,http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2001036,9.25,False,"[{'close_approach_date': '1910-02-25', 'close_...",False,http://www.neowsapp.com/rest/v1/neo/2001036?ap...,37.545248,83.953727,37545.247936,83953.726617,23.329528,52.166411,123179.951239,275438.744435,925,2021-05-21 05:50:06,1924-10-23,2021-05-20,35273,7047,0,.344347,3.034,2459396.5,.533129232790272,2.665548235434295,26.67564320070546,215.5328165431079,1589.564831738242,1.244466549711746,132.3995837190134,4.086629921156844,2458979.279223720400,94.4909427169498,.22

## Automatización de procesos

In [45]:
lista_1 =[1,2,3]

# El try siempre va acompañado con un except
# Es útil hacer esto para detectar el problema, y no parar la ejecución
# Es decir sale un error y eso nos para la ejecución, por lo que con try y except no borramos el error pero la ejecución continua en caso de haber un error
try:
  lista_1[10]
except:
  print("El índice esta fuera de rango")

El inidice esta fuera de rango


In [52]:
data_1 = {
    "column_1": [1,2,3],
    "column_2": [4,5,6]
}

df_1 = pd.DataFrame(data_1, index = ["a", "b", "c"])

data_2 = {
    "column_1": [7,8,9],
    "column_2": [10,11,12]
}

df_2 = pd.DataFrame(data_2, index = ["d", "e", "f"])
df_1

,column_1,column_2
a,1,4
b,2,5
c,3,6


In [53]:
# Concatenación
pd.concat([df_1, df_2], axis = 0)

,column_1,column_2
a,1,4
b,2,5
c,3,6
d,7,10
e,8,11
f,9,12


In [54]:
pd.concat([df_1, df_2], axis = 1)

,column_1,column_2,column_1,column_2
a,1.0,4.0,NaN,NaN
b,2.0,5.0,NaN,NaN
c,3.0,6.0,NaN,NaN
d,NaN,NaN,7.0,10.0
e,NaN,NaN,8.0,11.0
f,NaN,NaN,9.0,12.0


In [55]:
# Multindexación
df_concat = pd.concat([df_1,df_2], axis = 0, keys = ["df_1", "df_2"])
df_concat

column_1  column_2
df_1 a         1         4
     b         2         5
     c         3         6
df_2 d         7        10
     e         8        11
     f         9        12

In [56]:
# Podemos acceder al df multiindexado con loc
df_concat.loc[("df_1", "b")]

column_1    2
column_2    5
Name: (df_1, b), dtype: int64